Batchroom і dropout

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Клас MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_sizes, activations):
        super(MLP, self).__init__()
        layers = []

        # Додавання першого прихованого шару
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(self.get_activation(activations[0]))

        # Додавання наступних прихованих шарів
        for i in range(1, hidden_layers):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            layers.append(self.get_activation(activations[i]))

        # Додавання вихідного шару
        layers.append(nn.Linear(hidden_sizes[-1], 3))  # Кількість класів у Iris = 3

        self.model = nn.Sequential(*layers)

    def get_activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'sigmoid':
            return nn.Sigmoid()
        elif activation == 'tanh':
            return nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation: {activation}")

    def forward(self, x):
        return self.model(x)

# Завантаження і підготовка даних
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Завантаження даних Iris
iris = load_iris()
X = iris.data
y = iris.target

# Стандартизація даних
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Розбиття даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення датасетів та даталоадерів
train_dataset = IrisDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = IrisDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Конфігурація моделі
input_size = X_train.shape[1]
hidden_layers = 2
hidden_sizes = [16, 8]
activations = ['relu', 'relu']

model = MLP(input_size, hidden_layers, hidden_sizes, activations)

# Визначення критерію втрат та оптимізатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Навчальний цикл
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Оцінка моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Epoch 1/50, Loss: 1.0857714414596558
Epoch 2/50, Loss: 1.0653371438384056
Epoch 3/50, Loss: 1.0448299422860146
Epoch 4/50, Loss: 1.0145865678787231
Epoch 5/50, Loss: 0.9924337714910507
Epoch 6/50, Loss: 0.9724486842751503
Epoch 7/50, Loss: 0.9411404430866241
Epoch 8/50, Loss: 0.9107765555381775
Epoch 9/50, Loss: 0.8782017007470131
Epoch 10/50, Loss: 0.8447549715638161
Epoch 11/50, Loss: 0.8108634203672409
Epoch 12/50, Loss: 0.7650357261300087
Epoch 13/50, Loss: 0.7210005894303322
Epoch 14/50, Loss: 0.672670379281044
Epoch 15/50, Loss: 0.6295198053121567
Epoch 16/50, Loss: 0.6121366322040558
Epoch 17/50, Loss: 0.56797781214118
Epoch 18/50, Loss: 0.5349372029304504
Epoch 19/50, Loss: 0.5223607756197453
Epoch 20/50, Loss: 0.4967746026813984
Epoch 21/50, Loss: 0.48500682786107063
Epoch 22/50, Loss: 0.46621914952993393
Epoch 23/50, Loss: 0.4493495747447014
Epoch 24/50, Loss: 0.4440056663006544
Epoch 25/50, Loss: 0.4212651811540127
Epoch 26/50, Loss: 0.42162300273776054
Epoch 27/50, Loss: 0.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Клас MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_sizes, activations, batchnorm=False, dropout=False, dropout_probs=None):
        super(MLP, self).__init__()
        layers = []

        # Додавання першого прихованого шару
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        if batchnorm:
            layers.append(nn.BatchNorm1d(hidden_sizes[0]))
        layers.append(self.get_activation(activations[0]))
        if dropout and dropout_probs:
            layers.append(nn.Dropout(dropout_probs[0]))

        # Додавання наступних прихованих шарів
        for i in range(1, hidden_layers):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_sizes[i]))
            layers.append(self.get_activation(activations[i]))
            if dropout and dropout_probs:
                layers.append(nn.Dropout(dropout_probs[i]))

        # Додавання вихідного шару
        layers.append(nn.Linear(hidden_sizes[-1], 3))  # Кількість класів у Iris = 3

        self.model = nn.Sequential(*layers)

    def get_activation(self, activation):
        if activation == 'relu':
            return nn.ReLU()
        elif activation == 'sigmoid':
            return nn.Sigmoid()
        elif activation == 'tanh':
            return nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation: {activation}")

    def forward(self, x):
        return self.model(x)

# Завантаження і підготовка даних
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Завантаження даних Iris
iris = load_iris()
X = iris.data
y = iris.target

# Стандартизація даних
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Розбиття даних на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Створення датасетів та даталоадерів
train_dataset = IrisDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
test_dataset = IrisDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Конфігурація моделі
input_size = X_train.shape[1]
hidden_layers = 2
hidden_sizes = [16, 8]
activations = ['relu', 'relu']
batchnorm = True
dropout = True
dropout_probs = [0.5, 0.5]  # Ймовірності dropout для кожного шару

model = MLP(input_size, hidden_layers, hidden_sizes, activations, batchnorm, dropout, dropout_probs)

# Визначення критерію втрат та оптимізатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Навчальний цикл
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Оцінка моделі
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Epoch 1/50, Loss: 1.1479364782571793
Epoch 2/50, Loss: 1.1806806325912476
Epoch 3/50, Loss: 1.145462155342102
Epoch 4/50, Loss: 1.1234340220689774
Epoch 5/50, Loss: 1.134325660765171
Epoch 6/50, Loss: 1.0951309651136398
Epoch 7/50, Loss: 1.1183583810925484
Epoch 8/50, Loss: 1.0686556696891785
Epoch 9/50, Loss: 1.0763480216264725
Epoch 10/50, Loss: 1.0035307332873344
Epoch 11/50, Loss: 1.0917155370116234
Epoch 12/50, Loss: 1.0994360521435738
Epoch 13/50, Loss: 0.9678971320390701
Epoch 14/50, Loss: 0.9556282609701157
Epoch 15/50, Loss: 1.0282700210809708
Epoch 16/50, Loss: 0.9535896256566048
Epoch 17/50, Loss: 0.9789998978376389
Epoch 18/50, Loss: 0.9664704501628876
Epoch 19/50, Loss: 0.9723290279507637
Epoch 20/50, Loss: 0.8475498929619789
Epoch 21/50, Loss: 0.9597645103931427
Epoch 22/50, Loss: 1.0250433310866356
Epoch 23/50, Loss: 0.9202335551381111
Epoch 24/50, Loss: 0.9315709099173546
Epoch 25/50, Loss: 0.9720102250576019
Epoch 26/50, Loss: 0.9630486220121384
Epoch 27/50, Loss: 0.84